In [1]:
import sys

sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

## Import libraries

In [2]:
import tez
import os
import albumentations
import pandas as pd
import cv2
import numpy as np
import torch
import torch.nn as nn
import math
import random
import timm
import gc
import seaborn as sns

from sklearn import metrics
from tez.callbacks import EarlyStopping
from tqdm import tqdm

ONLY_SECOND_FOLD=True

In [3]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed) # if use multi-GPU

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(2021)
print("set seed")

In [4]:
df = pd.read_csv("../input/pet-remove-dup-ver2/train_remove_dup_10folds.csv")
#df = pd.read_csv("../input/same-old-creating-folds/train_10folds.csv")

dense_features = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]

In [5]:
class args:
    batch_size = 4
    image_size = 224
    epochs = 20
    fold = 10
    head_out = 192
    features = 0
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        
        # (720, 405, 3) -> (3, 720, 405)
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item] / 100.0
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

In [7]:
class PawpularModel(tez.Model):
    def __init__(self, model_name, train=True, head_out=128, features=12):
        super().__init__()
        
        self.model = timm.create_model(model_name, pretrained=train, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, head_out)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(head_out+features, 64)
        self.selu = nn.SELU()
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(64, 1)
        self.dense2.bias.data = torch.nn.Parameter(
            torch.Tensor([38.0])
        )
        
        self.step_scheduler_after = "epoch"

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy() #* 100.0
        targets = targets.cpu().detach().numpy() #* 100.0
        rmse = metrics.mean_squared_error(targets, outputs, squared=False)
        return {"rmse": rmse}

    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        
        #sch = torch.optim.lr_scheduler.CosineAnnealingLR(
        #    self.optimizer, T_max=10, eta_min=1e-6
        #)
        
        return sch

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-5)
        return opt

    def forward(self, image, features, targets=None):
        
        x = self.model(image) 
        #x = self.dropout(x1)
        #x = torch.cat([x, features], dim=1) 
        #x = self.selu(x)
        x = self.dense1(x) 
        #x = self.relu(x)
        x = self.dense2(x) 
        #x = self.selu(x)
        
        # x = torch.cat([x, x1, features], dim=1)

        if targets is not None:
            # RMSE
            # loss = nn.MSELoss()(x, targets.view(-1, 1))
            # metrics = self.monitor_metrics(x, targets)
            
            # BCE
            loss = nn.BCEWithLogitsLoss()(x, targets.view(-1, 1))
            metrics = self.monitor_metrics(torch.sigmoid(x) * 100, targets * 100)
            return x, loss, metrics
        
        return x, 0, {}

## aug all

In [ ]:
train_aug = albumentations.Compose(
    [
        #albumentations.RandomResizedCrop(
        #    height=args.image_size, width=args.image_size, 
        #    scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), p=0.5
        #),
     
        albumentations.Resize(args.image_size, args.image_size, p=1),
     
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
     
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
     
        albumentations.ShiftScaleRotate(
            shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2
        ),
     
        albumentations.HorizontalFlip(
            p=0.5
        ),
     
        albumentations.VerticalFlip(
            p=0.5
        ),
    
        albumentations.OneOf([
            albumentations.IAAAdditiveGaussianNoise(),
            albumentations.GaussNoise(),
        ], p=0.2),

        albumentations.OneOf([
            albumentations.MotionBlur(p=.2),
            albumentations.MedianBlur(blur_limit=3, p=0.1),
            albumentations.Blur(blur_limit=3, p=0.1),
        ], p=0.2),

        albumentations.OneOf([
            albumentations.OpticalDistortion(p=0.3),
            albumentations.GridDistortion(p=.1),
            albumentations.IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
     
        albumentations.OneOf([
            albumentations.CLAHE(clip_limit=2),
            albumentations.IAASharpen(),
            albumentations.IAAEmboss(),
            albumentations.RandomBrightnessContrast(),            
        ], p=0.3),

        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

## base aug

In [8]:
train_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        
        #albumentations.RandomResizedCrop(args.image_size, args.image_size, p=0.5),
        
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),

        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [ ]:
%%time

super_final_predictions = []

for fold_ in range(args.fold):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
    
    df_train = df[df.kfold != fold_].reset_index(drop=True)
    df_valid = df[df.kfold == fold_].reset_index(drop=True)
    
    train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]
    valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]
    
    train_dataset = PawpularDataset(
        image_paths=train_img_paths,
        dense_features=df_train[dense_features].values,
        targets=df_train.Pawpularity.values,#/100.0,
        augmentations=train_aug,
    )

    valid_dataset = PawpularDataset(
        image_paths=valid_img_paths,
        dense_features=df_valid[dense_features].values,
        targets=df_valid.Pawpularity.values,#/100.0,
        augmentations=valid_aug,
    )
    
    model = PawpularModel('swin_base_patch4_window7_224_in22k', train=True,
                           head_out=args.head_out, features=args.features)
    
    es = EarlyStopping(
        monitor="valid_rmse",
        model_path=f"model_f{fold_}.bin",
        patience=3,
        mode="min",
        save_weights_only=True,
    )

    model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs=args.batch_size,
        valid_bs=args.batch_size,
        device="cuda",
        epochs=args.epochs,
        callbacks=[es],
        fp16=True,
    )
    
    oof_predictions = model.predict(valid_dataset, batch_size=args.batch_size, n_jobs=-1)

    final_oof_predictions = []
    for preds in oof_predictions:
        final_oof_predictions.extend(preds.ravel().tolist())
    
    # BCE loss 
    final_oof_predictions = [sigmoid(x) * 100 for x in final_oof_predictions]
    super_final_predictions.append(final_oof_predictions)
    
    del df_train, df_valid, train_img_paths, valid_img_paths, train_dataset, valid_dataset
    gc.collect()
    
    if ONLY_SECOND_FOLD:
        if fold_ == 2:
            break

## CV

In [ ]:
df['oof'] = -1

for i in range(args.fold):
    df.loc[df['kfold']==i, 'oof'] = super_final_predictions[i]

    fold_preds = super_final_predictions[i]
    fold_target = df[df['kfold']==i]['Pawpularity']
    cv = np.sqrt((np.sum((fold_target - fold_preds)**2))/len(fold_preds))
    print(str(i+1)+' fold cv is ', np.round(cv, 5))
    
    if ONLY_SECOND_FOLD:
        if i == 2:
            break
        
if ONLY_SECOND_FOLD:
    pass
else:
    cv = np.sqrt((np.sum((df['Pawpularity'] - df['oof'])**2))/len(df['oof']))
    print('whole fold cv is ', np.round(cv, 5))

## Download

In [ ]:
!ls

In [ ]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'model_f0.bin')